In [1]:
#Cell 1: Install Required Libraries
!pip install groq duckduckgo-search pandas


Design and implement a ReAct (Reasoning + Acting) based Web Research Agent in Python using a Large Language Model (LLM) and a web search tool.

The system must:

Accept a research topic at runtime through a text input field in Jupyter.

Plan the research by using the LLM to generate 5–6 structured research questions.

Act on the plan by querying the web for each question using a search tool.

Extract and organize key insights from the retrieved information.

Synthesize a structured report in Markdown format containing:

Title

Introduction

Findings for each research question

Conclusion

The agent should follow the ReAct design pattern, where:

The LLM performs reasoning for planning and synthesis.

External tools perform actions for information retrieval.

The system produces a clean, noise-free output suitable for repeated execution with different topics.



In [3]:
#Cell 2: Imports + Warning Suppression + API Setup
import os
import warnings
import pandas as pd
from duckduckgo_search import DDGS
from groq import Groq

# Suppress all warnings
warnings.filterwarnings("ignore")

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "gsk"
client = Groq()


In [4]:
#Cell 3: Groq LLM Wrapper
def call_groq(prompt, model="llama-3.1-8b-instant", temperature=0.2):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a precise academic research assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()



In [5]:
#Cell 4: REASONING – Generate Research Questions
def generate_research_questions(topic):
    prompt = f"""
You are a research planning agent.

Topic: "{topic}"

Generate exactly 5 clear and well-structured research questions covering:
- Background
- Causes
- Trends / Data
- Impacts
- Policies or Solutions
- Future Outlook

Return only a numbered list. Do not add explanations.
"""
    response = call_groq(prompt)

    questions = []
    for line in response.split("\n"):
        line = line.strip()
        if line and line[0].isdigit():
            questions.append(line.split(".", 1)[1].strip())
    return questions


In [6]:
#Cell 5: ACTING STEP – Web Search Tool
def web_search(query, max_results=3):
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results):
            results.append({
                "title": r.get("title", ""),
                "snippet": r.get("body", ""),
                "source": r.get("href", "")
            })
    return results



In [7]:
#Cell 6: REASONING STEP – Extract Key Points from Search Results
def extract_key_points(question, search_results):
    search_text = ""
    for r in search_results:
        search_text += f"Title: {r['title']}\nSnippet: {r['snippet']}\n\n"

    prompt = f"""
You are an information extraction agent.

Research Question:
{question}

Search Results:
{search_text}

Task:
Extract 3 to 5 concise key points that directly answer the question.
Return only bullet points. Do not add commentary.
"""
    return call_groq(prompt)


In [8]:
#Cell 7: Internal ReAct Pipeline
def run_react_agent(topic):
    # Reasoning: Planning
    questions = generate_research_questions(topic)

    # Acting: Web Search + Extraction
    records = []
    for q in questions:
        search_results = web_search(q)
        key_points = extract_key_points(q, search_results)

        records.append({
            "Research Question": q,
            "Extracted Insights": key_points
        })

    df = pd.DataFrame(records)

    # Reasoning: Final Report
    table_text = df.to_string(index=False)

    prompt = f"""
You are a research report writing agent.

Topic: {topic}

Collected Research Data:
{table_text}

Task:
Write a structured research report in Markdown format with the following sections:

# Title
## Introduction
## Research Findings
- One subsection per research question.
- Summarize clearly and academically.
## Conclusion

Return only the report. Do not add explanations.
"""
    report = call_groq(prompt)
    return report



In [9]:
#Cell 8: Test Run – Topic 1
topic = "Climate Change"
report = run_react_agent(topic)
print(report)


/tmp/ipython-input-1282726859.py:4: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timez

# Climate Change: A Comprehensive Review of Drivers, Impacts, and Mitigation Efforts

## Introduction

Climate change is one of the most pressing issues of our time, with far-reaching consequences for ecosystems, human health, and the economy. This report provides a comprehensive review of the drivers of climate change, its impacts, and the most effective policy interventions and technological solutions for mitigating and adapting to its effects. The report is based on a thorough analysis of existing research and data, and aims to provide a clear understanding of the current state of climate change and its implications for the future.

## Research Findings

### Historical and Contemporary Drivers of Climate Change

The primary historical driver of climate change is the increase in greenhouse gases, particularly carbon dioxide (CO2), resulting from human activities such as deforestation, land-use changes, and the burning of fossil fuels, dating back to the Industrial Revolution (18th ce

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [17]:
#Cell 9: MAIN EXECUTION CELL (Instructor Will Run This)
# ===== RUNTIME INPUT =====
topic = input("Enter your research topic: ")

print("\n--- PLANNING PHASE: Generating Research Questions ---")
questions = generate_research_questions(topic)
for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

print("\n--- ACTING PHASE: Web Research ---")
df = build_research_table(questions)

print("\n--- REASONING PHASE: Report Generation ---")
final_report = generate_report(topic, df)

print("\n================ FINAL REPORT ================\n")
print(final_report)


Enter your research topic: AI

--- PLANNING PHASE: Generating Research Questions ---
1. What are the historical milestones and key developments in the field of Artificial Intelligence (AI), and how have they contributed to its current state of advancement?
2. What are the primary causes of the increasing adoption and integration of AI in various industries, and how have technological advancements, economic factors, and societal needs influenced this trend?
3. What are the current trends and statistics regarding the deployment and utilization of AI in different sectors, such as healthcare, finance, education, and transportation, and how are these trends likely to evolve in the future?
4. What are the significant impacts of AI on employment, social inequality, and individual autonomy, and how have these effects varied across different demographics and regions?
5. What are the existing policies and regulations governing the development and deployment of AI, and how effective are these mea

/tmp/ipython-input-3338443626.py:4: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timez

Researching: What are the primary causes of the increasing adoption and integration of AI in various industries, and how have technological advancements, economic factors, and societal needs influenced this trend?


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Researching: What are the current trends and statistics regarding the deployment and utilization of AI in different sectors, such as healthcare, finance, education, and transportation, and how are these trends likely to evolve in the future?


...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


--- REASONING PHASE: Report Generation ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


================ FINAL REPORT ================

# Artificial Intelligence: A Comprehensive Research Report

## Introduction

Artificial Intelligence (AI) has become a transformative force in modern society, revolutionizing various industries and aspects of human life. This research report aims to provide an in-depth examination of the historical milestones and key developments in AI, the causes of its increasing adoption and integration in different sectors, current trends and statistics regarding AI deployment, and the significant impacts of AI on employment, social inequality, and individual autonomy. Additionally, this report explores the existing policies and regulations governing AI development and deployment, as well as the potential future directions and applications of AI.

## Research Findings

### Historical Milestones and Key Developments in AI

Historical milestones and key developments in AI have significantly contributed to its current state of advancement. The Turing Te

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [11]:
topic = "Artificial Intelligence in Healthcare"
report = run_react_agent(topic)
print(report)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

# Artificial Intelligence in Healthcare: A Comprehensive Review

## Introduction

Artificial Intelligence (AI) has revolutionized the healthcare industry over the past decade, transforming the way healthcare services are delivered, and improving patient outcomes. This report provides a comprehensive review of the current state of AI in healthcare, including its applications, causes of adoption, trends, and impacts on healthcare outcomes. The report also discusses the most effective policies and solutions for ensuring the safe and equitable deployment of AI in healthcare.

## Research Findings

### What are the primary applications and areas of implementation of Artificial Intelligence (AI) in healthcare, and how have they evolved over the past decade?

The primary applications of AI in healthcare include medical imaging analysis, disease diagnosis, and personalized medicine. AI has been implemented in various areas such as electronic health records (EHRs), clinical decision support sys

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [12]:
# ===== RUNTIME INPUT (TEXTBOX) =====
topic = input("Enter your research topic: ")

report = run_react_agent(topic)

print("\n================ FINAL REPORT ================\n")
print(report)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replac

Enter your research topic: Agentic ai in real time


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


================ FINAL REPORT ================

# Agentic AI in Real-Time: A Comprehensive Analysis

## Introduction

Agentic AI, a subfield of artificial intelligence, has been gaining significant attention in recent years due to its potential to revolutionize various industries and domains. This report aims to provide a comprehensive analysis of the historical and theoretical underpinnings of agentic AI, its primary drivers and causal factors, current trends and patterns, potential impacts on human decision-making, and the necessary policies, regulations, and standards for its safe and transparent development and deployment.

## Research Findings

### Historical and Theoretical Underpinnings of Agentic AI

Agentic AI draws from philosophical theories of agency, such as Gilbert Ryle's 'The Concept of Mind' (1949) and John Searle's 'The Construction of Social Reality' (1995), which emphasize the importance of intentionality and self-awareness in intelligent systems. The development of

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag